# Import Libraries and Geometry


In [486]:
import ee, geemap, datetime, calendar, math

In [487]:
# Intialize GEE and create Interactive Map 
Map = geemap.Map(center=(11.43, 37.12), zoom=11)

In [488]:
# Import Geometry
Jedeb = ee.FeatureCollection("projects/ee-yilkalgebeyehu/assets/Jedeb_Catchment")

# Set Dates


In [489]:
# Set month and year number
m = 12
year = 2022

# Set date for satellite images
start = datetime.datetime(year, m, 1)
end = datetime.datetime(year, m, calendar.monthrange(year, m)[1])
LC_year_1 = datetime.datetime(year, 1, 1)
LC_year_2 = datetime.datetime(year, 12, 31)

# Constants


In [490]:
# Specific heat at constant pressure of air, kJ/kg K
c_p = 1.013
# Ratio molecular weight of water vapor/dry air
epsilon = 0.622
# Solar constant, kJ/m^2/h
a_s = 0.355
b_s = 0.68
I_s = 4921
# Albedo constant
a_e = 0.56
b_e = 0.08
a_l = 0.2
c2 = 90
# Conversion between Kelvin and Celsius
c3 = 273
# Stefan-Boltzmann constant, kJ/(m^2 K^4 d)
sigma = 0.0000049
coef = 0.33
# Other constants
cali = 0.2
LUE_max = 2.5
Kt = 23
Th = 35
Tl = 0
# Von karman constant
von = 0.41
# Albedo for Et0; refer to https://www.fao.org/3/x0490e/x0490e07.htm
Albedo = 0.25

# Import Climatic Datasets


In [491]:
climatic = ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY_AGGR").filter(ee.Filter.date(start, end))

climaticbands = climatic.toBands()

if m < 10:
    system = str(year) + "0" + str(m)
else:
    system = str(year) + str(m)

# Reduce Region
dict_T = climaticbands.select(system + "_" + "temperature_2m").reduceRegion(**{
        "reducer": ee.Reducer.mean(),
        "geometry": Jedeb.geometry(),
        "scale": 10000,
        "maxPixels": 1e9,
    }
)
# The result is a Dictionary.  Print it.
# print(dict_T.getInfo())

# Change the Temperature to celcius.
T_1 = dict_T.get(system + "_" + "temperature_2m")
T = ee.Number(T_1).subtract(c3)
# print(T.getInfo(), 'mean Temperature in C')
# print(T_1.getInfo(), 'mean Temperature in K')

# Reduce the region dewpoint temperature (Td)
dict_Td = climaticbands.select(system + "_" + "dewpoint_temperature_2m").reduceRegion(**{
        "reducer": ee.Reducer.mean(),
        "geometry": Jedeb.geometry(),
        "scale": 10000,
        "maxPixels": 1e9,
    }
)
# The result is a Dictionary.  Print it.
# print(dict_Td.getInfo())
Td = dict_Td.get(system + "_" + "dewpoint_temperature_2m")

# Calculating RH from dewpoint temperature
RH_1 = ee.Number(100).subtract(ee.Number(5).multiply(ee.Number(T_1).subtract(ee.Number(Td))))  
RH = RH_1.divide(ee.Number(100))
# print(RH_1.getInfo(), 'Mean Relative Humidity in %')

# Reduce the region Wind_speed (Both eastward and northward )
dict_Wu = climaticbands.select(system + "_" + "u_component_of_wind_10m").reduceRegion(**{
        "reducer": ee.Reducer.mean(),
        "geometry": Jedeb.geometry(),
        "scale": 10000,
        "maxPixels": 1e9,
    }
)

# The result is a Dictionary.  Print it.
Wu = dict_Wu.get(system + "_" + "u_component_of_wind_10m")

# Reduce the region Wind_speed (Both eastward and northward )
dict_Wv = climaticbands.select(system + "_" + "v_component_of_wind_10m").reduceRegion(**{
        "reducer": ee.Reducer.mean(),
        "geometry": Jedeb.geometry(),
        "scale": 10000,
        "maxPixels": 1e9,
    }
)

# The result is a Dictionary.  Print it.
Wv = dict_Wv.get(system + "_" + "v_component_of_wind_10m")

Wind_speed = ee.Number(Wu).multiply(ee.Number(Wu)).add(ee.Number(Wv).multiply(ee.Number(Wv))).sqrt()

# print(Wind_speed.getInfo(), 'mean Wind Speed in m/s')

In [492]:
# Import the NASA DEM ground elevation image.
elevation = ee.Image("NASA/NASADEM_HGT/001").select("elevation")
slope = ee.Terrain.slope(elevation)

# get coordinates image
latlon = ee.Image.pixelLonLat()

# Reduce the region. The region parameter is the Feature geometry.
dict_lat = latlon.select("latitude").reduceRegion(**{
        "reducer": ee.Reducer.mean(),
        "geometry": Jedeb.geometry(),
        "scale": 1000,
        "maxPixels": 1e9,
    }
)

# The result is a Dictionary.  Print it.
lat = dict_lat.get("latitude")
# print(dict_lat.getInfo())

# Reduce the region. The region parameter is the Feature geometry.
dict_lon = latlon.select("longitude").reduceRegion(**{
        "reducer": ee.Reducer.mean(),
        "geometry": Jedeb.geometry(),
        "scale": 1000,
        "maxPixels": 1e9,
    }
)

# The result is a Dictionary.  Print it.
long = dict_lon.get("longitude")
# print(dict_lon.getInfo())

# Reduce the region DEM
# Reduce the region. The region parameter is the Feature geometry.
dict_DEM = elevation.select("elevation").reduceRegion(**{
        "reducer": ee.Reducer.mean(),
        "geometry": Jedeb.geometry(),
        "scale": 30,
        "maxPixels": 1e9,
    }
)

# The result is a Dictionary.  Print it.
alt = dict_DEM.get("elevation")
# print(alt.getInfo())

In [493]:
# Map a function over MODIS
def getQABits(image, start, end, newName):
    # Compute the bits we need to extract.
    pattern = 0
    for i in range(start, end, 1):
        pattern += math.pow(2, i)

    return image.select([0], [newName]).bitwiseAnd(pattern).rightShift(start)

def clear(image):
    img = image.select("state_1km")
    return getQABits(img, 0, 1, "Clouds").expression("b(0) == 0 || b(0) == 3")

def get_cloudcover(year, month, geometry):
    geometry = ee.Geometry(geometry)

    # Load morning (Terra) MODIS data.
    morning = ee.ImageCollection("MODIS/061/MOD09GA").filterDate(start, end).filterBounds(Jedeb)
    
    clear_days = morning.map(clear)
    cloudcover = clear_days.reduce(ee.Reducer.mean()).rename("thres")
    return cloudcover


cloudcover = get_cloudcover(year, m, Jedeb)
# Map.addLayer(cloudcover,{min: 0, max: 100, palette: ['black','yellow']},'CLEAR DAYS %')

# Reduce the region. The region parameter is the Feature geometry.
dict_cloud = cloudcover.select("thres").reduceRegion(**{
        "reducer": ee.Reducer.mean(),
        "geometry": Jedeb.geometry(),
        "scale": 500,
        "maxPixels": 1e9,
    }
)

# The result is a Dictionary.  Print it.
sunduration_1 = dict_cloud.get("thres")

In [494]:
# Specific heat (kJ/kg) for a given temperature (C)
templamda = ee.Number(2501).subtract((ee.Number(2.361)).multiply(T))

# Standard pressure (hPa) for a given elevation (m)
temppressure_1 = ee.Number(alt).multiply(0.00002256)
temppressure_2 = ee.Number(1).subtract(temppressure_1)
temppressure_3 = temppressure_2.pow(5.256)
temppressure = temppressure_3.multiply(1013)

# Air density (kg/m^3) at a given air pressure (hPa) and temperature (oC)
tempdensity_1 = temppressure.multiply(0.3486)
tempdensity_2 = ee.Number(T).add(273)
tempdensity = tempdensity_1.divide(tempdensity_2)

# Psychrometric constant (hPa/C) for a given pressure (hPa) and Specific heat (kJ/kg)
tempgamma_1 = ee.Number(c_p).divide(ee.Number(epsilon))
tempgamma_2 = ee.Number(temppressure).divide(ee.Number(templamda))
tempgamma = ee.Number(tempgamma_1).multiply(ee.Number(tempgamma_2))

# Saturation vapor pressure (hPa) for a given temperature (C)
tempe_s_1 = ee.Number(T).add(237.3)
tempe_s_2 = ee.Number(T).multiply(17.27)
tempe_s_3 = tempe_s_2.divide(tempe_s_1)
tempe_s_4 = tempe_s_3.exp()
tempe_s = tempe_s_4.multiply(6.11)

# Slope of the saturation vapor pressure curve (hPa/C) for a given temperature (C)
tempdelta_1 = tempe_s_1.pow(2)
tempdelta = tempe_s.divide(tempdelta_1).multiply(4098)

# Astronomical quantities
if m == 1:
    j = 17
elif m == 2:
    j = 46
elif m == 3:
    j = 75
elif m == 4:
    j = 105
elif m == 5:
    j = 135
elif m == 6:
    j = 162
elif m == 7:
    j = 198
elif m == 8:
    j = 228
elif m == 9:
    j = 258
elif m == 10:
    j = 289
elif m == 11:
    j = 319
elif m == 12:
    j = 345

# Vapor pressure (hPa)
tempe = tempe_s.multiply(RH)

# Solar declination (rad) for a given day
tempdeclination = -0.4093 * math.cos(2 * 3.1415 * j / 365 + 0.16)

# Solar declination (degree) for a given day
tempdeclinationd = ee.Number(tempdeclination).multiply(180).divide(3.1415)

# Lat in radian
lat_rad = ee.Number(lat).multiply(3.1415).divide(180)

# Sun Duration
sunduration_2 = ee.Number(2 / 15)
sunduration_3 = (ee.Number(-1).multiply(ee.Number(tempdeclination))).tan().multiply(ee.Number(lat_rad).tan())
sunduration_4 = ee.Number(sunduration_3).acos()
sunduration_5 = ee.Number(sunduration_2).multiply(ee.Number(sunduration_4)).multiply(ee.Number(180)).divide(ee.Number(3.1415))

sunduration = ee.Number(sunduration_5).multiply(ee.Number(sunduration_1)).divide(ee.Number(24))

#print("Mean Sun Duration", sunduration.getInfo())

# Eccentricity for a given day
tempeccentricity = 1 + 0.034 * math.cos(2 * 3.1415 * j / 365 - 0.05)

# Sunset angle (rad) for a given day and latitude
tempphi_rad = ee.Number(lat).multiply(3.1415).divide(180)
tempphi = ee.Number(-1).multiply(ee.Number(tempphi_rad).tan()).multiply(ee.Number(tempdeclination).tan())

# For latitude > 66.5 (or < - 66.5)
tempo_s = tempphi
o_s = ee.Number.expression(
    "(b1 > 1.161) ? 0" + ": b2",
    {
        "b1": ee.Number(tempo_s).abs(),
        "b2": ee.Number(tempo_s).acos(),
    },
)

# Extraterrestial shortwave radiation (kJ/m^2/d) for a given day and latitude
S0temp = (24 / 3.1415) * I_s * tempeccentricity
tempS_0 = ee.Number(S0temp).multiply(ee.Number(o_s).multiply(ee.Number(tempphi_rad).sin())
    .multiply(ee.Number(tempdeclination).sin()).add(ee.Number(tempphi_rad).cos()
    .multiply(ee.Number(tempdeclination).cos()).multiply(ee.Number(o_s).sin())))

tempf_s = ee.Number(a_s).add((ee.Number(b_s)).multiply(ee.Number(1).subtract(ee.Number(sunduration))))
# Shortwave radiation (kJ/m^2/d) for given albedo, fraction of sunshine duration
tempS_n = (ee.Number(1).subtract(ee.Number(Albedo))).multiply(tempf_s).multiply(tempS_0)

# Lo
tempe_n_1 = tempe.sqrt()
tempe_n_2 = tempe_n_1.multiply(b_e)
tempe_n_3 = ee.Number(1).multiply(a_e)
tempe_n = tempe_n_3.subtract(tempe_n_2)

tempf_l = ee.Number(a_l).add(ee.Number(1).subtract(ee.Number(a_l))).multiply(ee.Number(1).subtract(ee.Number(sunduration)))

# Longwave radiation (kJ/m^2/d) for given temperature (C), fraction of sunshine duration
tempL_n_1 = tempe_n.multiply(tempf_l).multiply(sigma)
tempL_n_2 = ee.Number(T).add(c3)
tempL_n_3 = tempL_n_2.pow(4)
tempL_n = tempL_n_1.multiply(tempL_n_3)

# Mass transfer term for various cases of evaporation calculations
tempgamma_rc_1 = ee.Number(1).add(ee.Number(coef).multiply(ee.Number(Wind_speed)))
tempgamma_rc = tempgamma.multiply(tempgamma_rc_1)

# Mass transfer term (kg/(hPa m^2 d)) of reference crop for given wind speed (m/s)
tempF_rc_1 = ee.Number(T).add(c3)
tempF_rc_2 = ee.Number(1).multiply(c2)
tempF_rc = tempF_rc_2.divide(tempF_rc_1).multiply(ee.Number(Wind_speed))

# Penman-Montieth method
# A=Δ/(Δ+γ')
tempA_1 = tempdelta.add(tempgamma_rc)
tempA = tempdelta.divide(tempA_1)

# Β= γ/(Δ+γ')
tempB_1 = tempdelta.add(tempgamma_rc)
tempB = tempgamma.divide(tempB_1)

tempD = tempe_s.subtract(tempe)

tempRna = ee.Number(tempS_n).subtract(tempL_n)

temp_Epm_1 = tempA.multiply(tempRna)
temp_Epm_2 = tempB.multiply(tempF_rc).multiply(tempD)
temp_Epm_3 = templamda.add(temp_Epm_2)
temp_Epm = temp_Epm_1.divide(temp_Epm_3)

A = temp_Epm

if m == 1 or m == 3 or m == 5 or m == 7 or m == 8 or m == 10 or m == 12:
    monthday = 31
elif m == 2:
    monthday = 28
else:
    monthday = 30

ET0 = ee.Number(monthday).multiply(A)
print("Monthly Reference Evapotranspiration (mm)", ET0.getInfo())

Monthly Reference Evapotranspiration (mm) 96.32887953391437


# ETa from Satellite


In [495]:
def maskS2clouds(image):
    qa = image.select("QA60")

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask).divide(10000)


# Map the function over one year of data and take the median.
# Load Sentinel-2 TOA reflectance data.
collection = ee.ImageCollection("COPERNICUS/S2_HARMONIZED")\
        .filterDate(start, end)\
        .filterBounds(Jedeb)\
        .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20))\
        .map(maskS2clouds)

median = collection.median()

# Calculate median
s2m = median.select("B2", "B3", "B4", "B5", "B6", "B7", "B8", "B11", "B12")

# Calculate Albedo
albedo_img = s2m.expression(
    "(0.1324*b2)+(0.1269*b3)+(0.1051*b4)+(0.0971*b5)+(0.0890*b6)+(0.0818*b7)+(0.0722*b8)+(0.0167*b11)+(0.0002*b12)",
    {
        "b2": s2m.select("B2"),
        "b3": s2m.select("B3"),
        "b4": s2m.select("B4"),
        "b5": s2m.select("B5"),
        "b6": s2m.select("B6"),
        "b7": s2m.select("B7"),
        "b8": s2m.select("B8"),
        "b11": s2m.select("B11"),
        "b12": s2m.select("B12"),
    },
)

# Calculate NDVI
ndvi = s2m.normalizedDifference(["B8", "B4"])

In [496]:
# Total radiation (kJ/m^2/d) for given albedo, fraction of sunshine duration
rad_1 = albedo_img.multiply(-1)

rad_2 = rad_1.add(1)

tempS_na = rad_2.multiply(tempf_s).multiply(tempS_0)

tempRna = tempS_na.subtract(tempL_n)

# Total radiation (w/ha/month)
rna = tempRna.multiply(monthday).divide(864)

# Changing temperature with DEM
tdem = elevation.subtract(ee.Number(alt)).multiply(0.0065).add(ee.Number(T))

# Calculate G/Rn
GRn_1 = tdem.divide(albedo_img).multiply( albedo_img.multiply(0.0038).add((albedo_img.multiply(albedo_img).multiply(0.0074))))
GRn_2 = ndvi.multiply(ndvi.multiply(ndvi.multiply(ndvi.multiply(0.98))))
GRn_3 = GRn_2.multiply(-1).add(1)
GRn = GRn_1.multiply(GRn_3)

# Calculate soil heat flux
G = GRn.multiply(rna)

# Calculate LAI
LAI_1 = ndvi.multiply(ndvi.multiply(ndvi.multiply(9.519)))
LAI_2 = ndvi.multiply(ndvi.multiply(0.104))
LAI_3 = ndvi.multiply(1.236)
LAI = LAI_1.add(LAI_2).add(LAI_3).subtract(0.257)

In [497]:
LST = ee.ImageCollection("MODIS/061/MOD11A2").filterDate(start, end).median()
LSTm = LST.select("LST_Day_1km").multiply(0.02)

In [498]:
# Calculate surface roughness
zom = LAI.multiply(0.018)

# Calculate corrected zom
zom_1 = slope.subtract(5)
zom_2 = zom_1.divide(20).add(1)
zom_c = zom.multiply(zom_2)

# Calculate friction velocity
friction_velocity = ee.Number(von).multiply(ee.Number(Wind_speed)).divide(((ee.Number(2)).divide(ee.Number(0.0246))).log())

# Calculate wind speed at 200 m
wind_200_1 = ee.Number(200).divide(ee.Number(0.0246))
wind_200_2 = ee.Number(wind_200_1).log()
wind_200 = ee.Number(friction_velocity).multiply(wind_200_2).divide(ee.Number(von))

# Correct wind speed at 200 m
u200_1 = elevation.subtract(ee.Number(alt))
u200_2 = u200_1.divide(1000).multiply(0.1).add(1)
u200_c = u200_2.multiply(wind_200)

# A raster image with a value of 1
img_0 = ndvi.multiply(0)
img_1 = img_0.add(1)

# Calculate friction at 200 m
Fri_1 = u200_c.multiply(ee.Number(von))
twoh = img_1.multiply(200)
Fri_2 = twoh.divide(0.0246)
Fri_3 = Fri_2.log()
Fri_200 = Fri_1.divide(Fri_3)

# Calculate momemtum roughness
rah_1 = Fri_200.multiply(ee.Number(von))
rah_2 = img_1.multiply(2.995)
rah = rah_2.divide(rah_1)

In [499]:
# Find Agricultural Area (Created from Dynamic World land cover)
landcover = geemap.dynamic_world(Jedeb, LC_year_1, LC_year_2, clip=True, return_type="class")

cropland = landcover.expression(
    "(b1 == 4 || b1 == 2) ? 1" + ": 0",
    {
        "b1": landcover,
    },
)

cropland_c = cropland.expression(
    "(b1 == 1 && b2 > 0.20 ) ? 1" + ": 0",
    {
        "b1": cropland.select("constant"),
        "b2": ndvi.select("nd"),
    },
)

dem_cold = LSTm.multiply(cropland_c)

# Reduce Region
dem_coldmeandict = dem_cold.reduceRegion(**{
        "reducer": ee.Reducer.mean(),
        "geometry": Jedeb.geometry(),
        "scale": 10,
        "maxPixels": 1e9,
    }
)

# The result is a Dictionary.  Print it.
dem_coldmean = dem_coldmeandict.get("LST_Day_1km")

dem_coldmean_img = img_1.multiply(dem_coldmean)

dem_coldmeandict1 = dem_cold.reduceRegion(**{
        "reducer": ee.Reducer.stdDev(),
        "geometry": Jedeb.geometry(),
        "scale": 10,
        "maxPixels": 1e9,
    }
)

# The result is a Dictionary.  Print it.
dem_coldstd = dem_coldmeandict1.get("LST_Day_1km")

thre_cold = ee.Number(dem_coldmean).add(ee.Number(dem_coldstd).multiply(2))

precold_1 = LSTm.expression(
    "(b1 < b2) ? 1" + ": 0",
    {
        "b1": LSTm.select("LST_Day_1km"),
        "b2": thre_cold,
    },
)

precold = precold_1.multiply(cropland_c)
# print(precold.getInfo())

In [500]:
# Specific heat (kJ/kg) for a given temperature (C)
templamdaa_1 = tdem.multiply(2.361)
templamdaa = img_1.multiply(2501).subtract(templamdaa_1)

# Standard pressure (hPa) for a given elevation (m)
temppressurea_1 = elevation.multiply(0.00002256)
temppressurea_2 = img_1.subtract(temppressurea_1)
temppressurea_3 = temppressurea_2.pow(5.256)
temppressurea = temppressurea_3.multiply(1013)

# Air density (kg/m^3) at a given air pressure (hPa) and temperature (C)
tempdensitya_1 = temppressurea.multiply(0.3486)
tempdensitya_2 = tdem.add(273)
tempdensitya = tempdensitya_1.divide(tempdensitya_2)

# Psychrometric constant (hPa/C) for a given pressure (hPa) and Specific heat (kJ/kg)
tempgammaa_1 = temppressurea.multiply(ee.Number(c_p).divide(ee.Number(epsilon)))
tempgammaa = tempgammaa_1.divide(templamdaa)

# Saturation vapor pressure (hPa) for a given temperature (C)
tempe_sa_1 = tdem.add(237.3)
tempe_sa_2 = tdem.multiply(17.27)
tempe_sa_3 = tempe_sa_2.divide(tempe_sa_1)
tempe_sa_4 = tempe_sa_3.exp()
tempe_sa = tempe_sa_4.multiply(6.11)

# Slope of the saturation vapor pressure curve (hPa/C) for a given temperature (C)
tempdeltaa_1 = tempe_sa_1.pow(2)
tempdeltaa = tempe_sa.divide(tempdeltaa_1).multiply(4098)

# Vapor pressure (hPa)
tempea = tempe_sa.multiply(ee.Number(RH))

# Lo
tempe_na_1 = tempea.sqrt()
tempe_na_2 = tempe_na_1.multiply(b_e)
tempe_na_3 = img_1.multiply(a_e)
tempe_na = tempe_na_3.subtract(tempe_na_2)

# Longwave radiation (kJ/m^2/d) for given temperature (C), fraction of sunshine duration
tempL_na_1 = tempe_na.multiply(tempf_l).multiply(sigma)
tempL_na_2 = tdem.add(c3)
tempL_na_3 = tempL_na_2.pow(4)
tempL_na = tempL_na_1.multiply(tempL_na_3)

# Mass transfer term for various cases of evaporation calculations
tempgamma_rca_1 = ee.Number(1).add(ee.Number(coef).multiply(ee.Number(Wind_speed)))
tempgamma_rca = tempgammaa.multiply(tempgamma_rca_1)

# Mass transfer term (kg/(hPa m^2 d)) of reference crop for given wind speed (m/s)
tempF_rca_1 = tdem.add(c3)
tempF_rca_2 = img_1.multiply(c2)
tempF_rca = tempF_rca_2.divide(tempF_rca_1).multiply(ee.Number(Wind_speed))

# Penman-Montieth method
# A=Δ/(Δ+γ')
tempAa_1 = tempdeltaa.add(tempgamma_rca)
tempAa = tempdeltaa.divide(tempAa_1)

# Β= γ/(Δ+γ')
tempBa_1 = tempdeltaa.add(tempgamma_rca)
tempBa = tempgammaa.divide(tempBa_1)

tempDa = tempe_sa.subtract(tempea)

tempRnaa = tempS_na.subtract(tempL_na)

temp_Epma_1 = tempAa.multiply(tempRnaa)
temp_Epma_2 = tempBa.multiply(tempF_rca).multiply(tempDa)
temp_Epma_3 = templamdaa.add(temp_Epma_2)
temp_Epma = temp_Epma_1.divide(temp_Epma_3)

# Calculate ET0s
ET0s = temp_Epma

# Continue with cold pixels calculations
rastercold_1 = rna.subtract(G)
rastercold_2 = ET0s.multiply(1.05)
rastercold = rastercold_1.subtract(rastercold_2).multiply(precold)

# Reduce the region
dem_coldmeandict1 = rastercold.reduceRegion(**{
        "reducer": ee.Reducer.mean(),
        "geometry": Jedeb.geometry(),
        "scale": 10,
        "maxPixels": 1e9,
    }
)

# The result is a Dictionary.  Print it.
H_cold = dem_coldmeandict1.get("constant")

raster_cold = rah.multiply(ee.Number(H_cold)).divide(tempdensitya.multiply(1004))

dt_cold_img = raster_cold.multiply(precold)

# Reduce the region
dt_coldmeandict1 = dt_cold_img.reduceRegion(**{
        "reducer": ee.Reducer.mean(),
        "geometry": Jedeb.geometry(),
        "scale": 10,
        "maxPixels": 1e9,
    }
)

# The result is a Dictionary.  Print it.
dt_cold = dt_coldmeandict1.get("nd")

lst_dt_img_c = LSTm.multiply(precold)

# Reduce the region
dt_coldlst = lst_dt_img_c.reduceRegion(
    **{
        "reducer": ee.Reducer.mean(),
        "geometry": Jedeb.geometry(),
        "scale": 10,
        "maxPixels": 1e9,
    }
)

# The result is a Dictionary.  Print it.
lst_dt_cold = dt_coldlst.get("LST_Day_1km")

In [501]:
# Reterieve hot pixels
cropland_h = cropland.expression(
    "(b1 < 0.20 && b1 > 0 && b2 == 1) ? 1" + ": 0",
    {
        "b1": ndvi.select("nd"),
        "b2": cropland.select("constant"),
    },
)

dem_hot = LSTm.multiply(cropland_h)

# Reduce the region
dem_hotmeandict = dem_hot.reduceRegion(**{
        "reducer": ee.Reducer.mean(),
        "geometry": Jedeb.geometry(),
        "scale": 10,
        "maxPixels": 1e9,
    }
)

# The result is a Dictionary.  Print it.
dem_hotmean = dem_hotmeandict.get("LST_Day_1km")

# Reduce the region
dem_hotmeandict1 = dem_hot.reduceRegion(**{
        "reducer": ee.Reducer.stdDev(),
        "geometry": Jedeb.geometry(),
        "scale": 10,
        "maxPixels": 1e9,
    }
)
# The result is a Dictionary.  Print it.
dem_hotstd = dem_hotmeandict.get("LST_Day_1km")

thre_hot = ee.Number(dem_hotmean).subtract(ee.Number(dem_hotstd))

prehot_1 = LSTm.expression(
    "(b1 > b2) ? 1" + ": 0",
    {
        "b1": LSTm.select("LST_Day_1km"),
        "b2": thre_hot,
    },
)

prehot = prehot_1.multiply(cropland_h)

rasterhot_1 = rna.subtract(G)
rasterhot = rastercold_1.multiply(prehot)

# Reduce the region
dem_hotmeandict1 = rasterhot.reduceRegion(**{
        "reducer": ee.Reducer.mean(),
        "geometry": Jedeb.geometry(),
        "scale": 10,
        "maxPixels": 1e9,
    }
)

# The result is a Dictionary.  Print it.
H_hot = dem_hotmeandict1.get("constant")

raster_hot = rah.multiply(ee.Number(H_hot)).divide(tempdensitya.multiply(1004))

lst_dt_img_h = raster_hot.multiply(prehot)

# Reduce the region
dt_hotmeandict1 = lst_dt_img_h.reduceRegion(**{
        "reducer": ee.Reducer.mean(),
        "geometry": Jedeb.geometry(),
        "scale": 10,
        "maxPixels": 1e9,
    }
)

# The result is a Dictionary.  Print it.
dt_hot = dt_hotmeandict1.get("nd")

lst_dt_img_h = LSTm.multiply(prehot)

# Reduce the region
dt_hotlst = lst_dt_img_h.reduceRegion(**{
        "reducer": ee.Reducer.mean(),
        "geometry": Jedeb.geometry(),
        "scale": 10,
        "maxPixels": 1e9,
    }
)

# The result is a Dictionary.  Print it.
lst_dt_hot = dt_hotlst.get("LST_Day_1km")

# Find a and b
a_dt = ee.Number(dt_hot).subtract(ee.Number(dt_cold)).divide((ee.Number(lst_dt_hot).subtract(ee.Number(lst_dt_cold))))

b_dt = ee.Number(dt_hot).subtract((ee.Number(a_dt).multiply(ee.Number(lst_dt_hot))))

# Calculate dT
dT = LSTm.multiply(a_dt).add(b_dt)
# Calculate sensible heat flux
H_1 = dT.multiply(tempdensity).multiply(1004).divide(rah)

In [502]:
# Water pixels sensible heat
water_qa = landcover.expression(
    "(b1 == 0) ? 1" + ": 0",
    {
        "b1": landcover,
    },
)

waterH1 = H_1.multiply(water_qa)

# Reduce the region
dict_w1 = waterH1.reduceRegion(**{
        "reducer": ee.Reducer.mean(),
        "geometry": Jedeb.geometry(),
        "scale": 10,
        "maxPixels": 1e9,
    }
)
# The result is a Dictionary.  Print it.
meanhw1 = dict_w1.get("LST_Day_1km")

if RH.lt(0.4) and ee.Number(Wind_speed).gt(2):
    kpan = 1.509433962
if RH.lt(0.4) and ee.Number(Wind_speed).gt(2) and ee.Number(Wind_speed).lt(5):
    kpan = 1.632653061
if RH.lt(0.4) and ee.Number(Wind_speed).gt(5) and ee.Number(Wind_speed).lt(8):
    kpan = 1.777777778
if RH.lt(0.4) and ee.Number(Wind_speed).gt(8):
    kpan = 2.105263158
if RH.gt(0.4) and RH.lt(0.7) and ee.Number(Wind_speed).lt(2):
    kpan = 1.31147541
if  RH.gt(0.4) and RH.lt(0.7) and ee.Number(Wind_speed).gt(2) and ee.Number(Wind_speed).lt(5):
    kpan = 1.403508772
if  RH.gt(0.4) and RH.lt(0.7) and ee.Number(Wind_speed).gt(5) and ee.Number(Wind_speed).lt(8):
    kpan = 1.632653061
if RH.gt(0.4) and RH.lt(0.7) and ee.Number(Wind_speed).gt(8):
    kpan = 1.818181818
if RH.gt(0.7) and ee.Number(Wind_speed).lt(2):
    kpan = 1.212121212
if RH.gt(0.7) and ee.Number(Wind_speed).gt(2) and ee.Number(Wind_speed).lt(5):
    kpan = 1.333333333
if RH.gt(0.7) and ee.Number(Wind_speed).gt(5) and ee.Number(Wind_speed).lt(8):
    kpan = 1.481481481
if RH.gt(0.7) and ee.Number(Wind_speed).gt(8):
    kpan = 1.666666667
else:
    kpan = 1.620253165

waterH2_1 = ET0s.multiply(kpan).multiply(cali)
waterH2 = rna.subtract(waterH2_1).multiply(water_qa)

# Reduce the region
dict_w2 = waterH2.reduceRegion(**{
        "reducer": ee.Reducer.mean(),
        "geometry": Jedeb.geometry(),
        "scale": 10,
        "maxPixels": 1e9,
    }
)

# The result is a Dictionary.  Print it.
meanhw2 = dict_w2.get("constant")

# Calculate surface roughness of grass
rah_grass = ee.Number(208).divide(ee.Number(Wind_speed))

# Reduce the region
dict_dtw = LSTm.reduceRegion(**{
        "reducer": ee.Reducer.stdDev(),
        "geometry": Jedeb.geometry(),
        "scale": 10,
        "maxPixels": 1e9,
    }
)

# The result is a Dictionary.  Print it.
meandtw_1 = dict_dtw.get("LST_Day_1km")
meandtw = ee.Number(meandtw_1).multiply(4)

waterH3 = tempdensitya.multiply(1004).multiply(meandtw).divide(rah_grass)

# Reduce the region
dict_w3 = waterH3.reduceRegion(**{
        "reducer": ee.Reducer.stdDev(),
        "geometry": Jedeb.geometry(),
        "scale": 10,
        "maxPixels": 1e9,
    }
)

# The result is a Dictionary.  Print it.
meanhw3 = dict_w3.get("nd")

meanhw23 = ee.Number(meanhw2).max(ee.Number(meanhw3))

meanhwnew = ee.Number(meanhw23).subtract(ee.Number(meanhw1))

# Calculate new sensible heat
H = H_1.add(ee.Number(meanhwnew))

# Retreive water pixels latent heat
water_ET0s = water_qa.multiply(ET0s)

# Reduce the region
dict_et0s = water_ET0s.reduceRegion(**{
        "reducer": ee.Reducer.mean(),
        "geometry": Jedeb.geometry(),
        "scale": 10,
        "maxPixels": 1e9,
    }
)

# The result is a Dictionary.  Print it.
meanet0sw = dict_et0s.get("constant")

# Calculate LET
LET_1 = rna.subtract(G).subtract(H)

water_LET = water_qa.multiply(LET_1)

# Reduce the region
dict_LET = water_LET.reduceRegion(**{
        "reducer": ee.Reducer.mean(),
        "geometry": Jedeb.geometry(),
        "scale": 10,
        "maxPixels": 1e9,
    }
)

# print(dict_LET.getInfo())

In [503]:
# The result is a Dictionary.  Print it.
meanLETw = dict_LET.get("constant")

dw_LET = (ee.Number(meanet0sw).multiply(kpan).multiply(cali)).subtract(ee.Number(meanLETw))

# Calculate LET
LET = LET_1.expression(
    "(b1 + b2 < 0) ? 0.01" + ": b1 + b2", {"b1": LET_1.select("constant"), "b2": dw_LET}
)

# Calculate EF
EF_1 = rna.subtract(G)
EF = LET.divide(EF_1)

# Saturation Vapor Pressure at the air temperature (kPa):
esat_1 = tdem.multiply(17.27)
esat_2 = tdem.add(237.3)
esat_3 = esat_1.divide(esat_2)
esat_4 = esat_3.exp()
esat = esat_4.multiply(0.6108)

#  Actual vapour pressure (kPa), FAO 56, eq 19.:
eact = esat.multiply(RH).divide(100)

# Advection factor
AF_1 = esat.subtract(eact)
AF_2 = AF_1.multiply(0.08).subtract(1)
AF_3 = AF_2.exp()
AF_4 = AF_3.multiply(EF).multiply(0.985)
AF = AF_4.add(1)
# Calculate ETa from satellite
ETas = EF.multiply(rna).multiply(AF).divide(28.356).divide(245000000).multiply(86400000)

# Calculate Kc
Kc = ETas.divide(ET0s)

# Calculate new ETa
ETa_1 = Kc.multiply(ET0).subtract(2.089).divide(1.0864)
ETa = ETa_1.multiply(1.0117).add(11.28)

# Water Productivity


In [504]:
# Calculate PAR
PAR = tempL_na.divide(8.64).multiply(0.48)

# Calculate FPAR
FPAR_1 = ndvi.expression(
    "(b1 < 0.125 ) ? 0" + ": b1",
    {
        "b1": ndvi.select("nd"),
    },
)

FPAR = FPAR_1.subtract(0.161).add(1.257)

# Calculate APAR
APAR = PAR.multiply(FPAR)

# Calculate vapor stress scalar
v_stress_1 = esat.subtract(eact)
v_stress_2 = v_stress_1.log()
v_stress_3 = v_stress_2.multiply(0.183)
v_stress_4 = img_1.multiply(0.88)
v_stress = v_stress_4.subtract(v_stress_3)

# Calculate heat stress scalar
Jarvis_coeff = (Th - Kt) / (Kt - Tl)

h_stress_1 = tdem.subtract(Tl)

h_stress_2 = img_1.multiply(Th)
h_stress_3 = h_stress_2.subtract(tdem)
h_stress_4 = h_stress_3.pow(Jarvis_coeff)

h_stress_5 = (Kt - Tl) * math.pow(Th - Kt, Jarvis_coeff)

h_stress = h_stress_1.multiply(h_stress_4).divide(h_stress_5)

# Calculate moisture stress scalar
m_stress = LET.divide((LET).add(H))

# Calculate LUE
LUE = v_stress.multiply(h_stress).multiply(m_stress).multiply(LUE_max)

# Caluclate biomass production
biomass_prod = APAR.multiply(LUE).multiply(0.864)

# Calculate water productivity from satellite
WPs = biomass_prod.divide((ETas).multiply(10))

# Calculate net biomass production
biomass_pod_n = ETa.multiply(WPs).multiply(10)

# Calculate water productivity
WP = biomass_pod_n.divide((ETa).multiply(10)).multiply(cropland)

# Statistics


In [505]:
# Reduce the region. The region parameter is the Feature geometry
meanDictionary = ETa.reduceRegion(**{
        "reducer": ee.Reducer.mean(),
        "geometry": Jedeb.geometry(),
        "scale": 10,
        "maxPixels": 1e9,
    }
)

# The result is a Dictionary.  Print it.
meaneta = meanDictionary.get("constant")
print(system, "Mean ETa (mm)", meaneta.getInfo())

# Reduce the region. The region parameter is the Feature geometry
meanDictionarystd = ETa.reduceRegion(**{
        "reducer": ee.Reducer.stdDev(),
        "geometry": Jedeb.geometry(),
        "scale": 10,
        "maxPixels": 1e9,
    }
)

# The result is a Dictionary.  Print it.
stdeta = meanDictionarystd.get("constant")
# print("Std ETa (mm)", stdeta.getInfo())

# Reduce the region. The region parameter is the Feature geometry
meanDictionary1 = Kc.reduceRegion(**{
        "reducer": ee.Reducer.mean(),
        "geometry": Jedeb.geometry(),
        "scale": 10,
        "maxPixels": 1e9,
    }
)

# The result is a Dictionary.  Print it.
meanet0 = meanDictionary1.get("constant")
# print('Mean Kc', meanet0.getInfo())

# Reduce the region. The region parameter is the Feature geometry
meanbiomass = biomass_pod_n.reduceRegion(**{
        "reducer": ee.Reducer.mean(),
        "geometry": Jedeb.geometry(),
        "scale": 10,
        "maxPixels": 1e9,
    }
)

# The result is a Dictionary.  Print it.
meanbio = meanbiomass.get("constant")
# print('Mean Biomass Production (kg/m3)', meanbio.getInfo())

# Reduce the region. The region parameter is the Feature geometry
stdbiomass = biomass_pod_n.reduceRegion(**{
        "reducer": ee.Reducer.stdDev(),
        "geometry": Jedeb.geometry(),
        "scale": 10,
        "maxPixels": 1e9,
    }
)

# The result is a Dictionary.  Print it.
stdbio = stdbiomass.get("constant")
# print('Std Biomass Production (kg/m3)', stdbio.getInfo())

# Reduce the region. The region parameter is the Feature geometry
meanWP = WP.reduceRegion(**{
        "reducer": ee.Reducer.mean(),
        "geometry": Jedeb.geometry(),
        "scale": 10,
        "maxPixels": 1e9,
    }
)

# The result is a Dictionary.  Print it.
meanwp = meanWP.get("constant")
# print('Mean Water Productivity (kg/m3)', meanwp.getInfo())

# Reduce the region. The region parameter is the Feature geometry
stdWP = WP.reduceRegion(**{
        "reducer": ee.Reducer.stdDev(),
        "geometry": Jedeb.geometry(),
        "scale": 10,
        "maxPixels": 1e9,
    }
)

# The result is a Dictionary.  Print it.
stdwp = stdWP.get("constant")
# print('Std Water Productivity (kg/m3)', stdwp.getIfo())

# Reduce the region. The region parameter is the Feature geometry
meanDictionaryaf = H_1.reduceRegion(**{
        "reducer": ee.Reducer.mean(),
        "geometry": Jedeb.geometry(),
        "scale": 10,
        "maxPixels": 1e9,
    }
)

# The result is a Dictionary.  Print it.
meanaf = meanDictionaryaf.get("LSTm")
# print('Mean rah', meanaf.getInfo())

# Reduce the region. The region parameter is the Feature geometry
meanDictionaryas = ETas.reduceRegion(**{
        "reducer": ee.Reducer.mean(),
        "geometry": Jedeb.geometry(),
        "scale": 10,
        "maxPixels": 1e9,
    }
)

# The result is a Dictionary.  Print it.
meanas = meanDictionaryas.get("constant")
# print('Mean ETas', meanas.getInfo())

# Reduce the region. The region parameter is the Feature geometry
meanDictionarylet = rna.reduceRegion(**{
        "reducer": ee.Reducer.mean(),
        "geometry": Jedeb.geometry(),
        "scale": 10,
        "maxPixels": 1e9,
    }
)

# The result is a Dictionary.  Print it.
meanlet = meanDictionarylet.get("constant")
# print('Mean rna', meanlet.getInfo())

202212 Mean ETa (mm) 92.25608738224255


# Blue and Green Water Consumption


In [221]:
# Import CHIRPS daily preciptation
PCP = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY").select("precipitation").filterDate(start, end)

# Calculate the monthly precipitation
Pmonth = PCP.reduce(ee.Reducer.sum())


# Define a function to calculate effective rainfall (USDA SCS Formula)
def effective_rainfall(Pmonth):
    Pe = Pmonth.expression(
        "p <= 250 ? p * (125 - 0.2 * p) / 125 : 125 + 0.1 * p", {"p": Pmonth}
    )
    return Pe

# Calculate effective rainfall
Pe = effective_rainfall(Pmonth)

In [222]:
# Calculate geen and blue water consumption, Green ETa and Blue ETa
    # Green ETa = min (Pe, ETa)
    # Blue ETa = max (0, ETa - Pe), where ETa is actual evapotranpiration and Pe is effective precipitaiton

GreenETa_1 = Pe.min(ETa)

# Create a binary mask where water pixels are 0 and non-water pixels are 1
waterMask = water_qa.eq(0)

GreenETa = GreenETa_1.updateMask(waterMask)  # To mask water bodies from GreenETa

BlueETa = ETa.subtract(Pe).max(0)

In [223]:
# Reduce the region. The region parameter is the Feature geometry
meanDictionary = GreenETa.reduceRegion(**{
        "reducer": ee.Reducer.mean(),
        "geometry": Jedeb.geometry(),
        "scale": 10,
        "maxPixels": 1e9,
    }
)

# The result is a Dictionary.  Print it.
meangwe = meanDictionary.get("constant")
print(system, "Green Eta (mm)", meangwe.getInfo())

# Reduce the region. The region parameter is the Feature geometry
meanDictionary = BlueETa.reduceRegion(**{
        "reducer": ee.Reducer.mean(),
        "geometry": Jedeb.geometry(),
        "scale": 10,
        "maxPixels": 1e9,
    }
)

# The result is a Dictionary.  Print it.
meanbwe = meanDictionary.get("constant")
print(system, "Blue Eta (mm)", meanbwe.getInfo())

202005 Green Eta (mm) 109.71719641115483
202005 Blue Eta (mm) 26.846573861945018


# Visualize Results


In [224]:
vis_cover = {
    "min": 0,
    "max": 8,
    "palette": ["#419BDF","#397D49","#88B053","#7A87C6","#E49635","#DFC35A","#C4281B","#A59B8F","#B39FE1"],
}

vis_natural = {'bands': ['B4', 'B3', 'B2'], 'min': 0.0,  'max': 0.3}

ndviViz = {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}

visParams_eta = {'min': 0, 'max': 200, 'palette': ["black", "blue", "cyan", "green", "yellow", "red"]}

Map.addLayer(Jedeb, {}, "Jedeb", False)
Map.addLayer(s2m, vis_natural, 'True color (432)', False)
Map.addLayer(ndvi,ndviViz,'Sentinel-2 NDVI', False)
Map.addLayer(landcover, vis_cover, "Land Cover", False)
Map.addLayer(ETa, visParams_eta, "Sentinel-2 ETa", False)
Map.addLayer(Pe, {"min": 0, "max": 100}, "Effective Rainfall", False)
Map.addLayer(GreenETa, visParams_eta, "Green ETa", False)
Map.addLayer(BlueETa, visParams_eta, "Blue ETa", False)
Map

Map(center=[11.43, 37.12], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childre…

# Export Result


In [225]:
roi = Jedeb.geometry()